In [2]:
import tensorflow
tf = tensorflow.compat.v1
tf.disable_eager_execution()

In [3]:
n_inputs = 3
n_neurons = 5

X0 = tf.placeholder(dtype=tf.float32, shape=[None, n_inputs])
X1 = tf.placeholder(dtype=tf.float32, shape=[None, n_inputs])

Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)

init = tf.global_variables_initializer()

In [4]:
import numpy as np

# Mini-batch:  4 instances
X0_batch = np.array([
    [0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]
])
X1_batch = np.array([
    [9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]
])

在之前定义tf.variable里的W时只是定义了这个变量值初始化的方式的，但并没有真正对其初始化，直到运行了tf.global_variables_initializer().run()之后才完成了变量的初始化。
sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch}) 的意思是执行[Y0, Y1]函数，执行时将其中的X0和X1替换为X0_batch, X1_batch。
下面的函数只是执行了一步，还并没有开始训练

In [12]:
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [13]:
print(Y0_val)
print(Y1_val)

[[ 0.9933106  -0.53062135  0.823383   -0.99261296 -0.20229085]
 [ 0.99190515 -0.99763733  0.89730775 -1.         -0.99029636]
 [ 0.9902062  -0.99999094  0.94128704 -1.         -0.99992836]
 [-1.          0.48571843 -1.         -0.99999917 -0.99999946]]
[[-0.9885508  -1.          0.9484859  -1.         -0.99894595]
 [ 0.6136913  -0.22664668  0.99953645 -0.9186043   0.99823034]
 [-0.98015636 -0.9999363   0.99858886 -1.         -0.8355101 ]
 [-0.98742557 -0.5547205  -0.9501999  -0.99999505 -0.83876127]]


In [16]:
X0 = tf.placeholder(dtype=tf.float32, shape=[None, n_inputs])
X1 = tf.placeholder(dtype=tf.float32, shape=[None, n_inputs])

# basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
basic_cell = tensorflow.keras.layers.SimpleRNNCell(units=n_neurons)
output_seqs, states = tf.nn.static_rnn(cell=basic_cell, inputs=[X0, X1], dtype=tf.float32)

Y0, Y1 = output_seqs

上面这种方法虽然可行，但显然当时间迭代长的时候是一件不那么容易的事情，所以，我们换一种做法。

tf.transpose(X, perm=[1, 0, 2]) 是一个张量转置的函数，在这里就是将第一维和第二维转置，第三维保持不变。交换维度后，第一个时间迭代将称为第一个维度。unstack函数将抽取第一个维度的张良；stack函数合并所有输出张量为一个张量

In [18]:
n_steps = 2

X = tf.placeholder(dtype=tf.float32, shape=[None, n_steps, n_inputs])
X_seqs = tf.unstack(tf.transpose(X, perm=[1, 0, 2]))
basic_cell = tensorflow.keras.layers.SimpleRNNCell(units=n_neurons)
output_seqs, states = tf.nn.static_rnn(cell=basic_cell, inputs=X_seqs, dtype=tf.float32)

outputs = tf.transpose(tf.stack(output_seqs), perm=[1, 0, 2])   # shape [None, n_steps, n_neurons]

In [19]:
X_batch = np.array([
    [[0, 1, 2], [9, 8, 7]],
    [[3, 4, 5], [0, 0, 0]], 
    [[6, 7, 8], [6, 5, 4]], 
    [[9, 0, 1], [3, 2, 1]]
])

In [22]:
with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})

FailedPreconditionError: Graph execution error:

Detected at node 'rnn_2/simple_rnn_cell_1/MatMul_1/ReadVariableOp' defined at (most recent call last):
    File "d:\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "d:\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "d:\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "d:\Python\Python310\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "d:\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "d:\Python\Python310\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "d:\Python\Python310\lib\asyncio\base_events.py", line 600, in run_forever
      self._run_once()
    File 